In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
def create_graph(df) -> dict:
    """
    returns a didct with keys = node key, and values = edges (node_id, edge weight) 
    """
    tree = {}
    for _, row in df.iterrows():
        parent = row['Parent']
        child = row['Child']
        weight = row['t']

        # root node
        if pd.isna(parent): 
            continue
        # node already exists
        elif parent in tree: 
            tree[parent].append((child, weight))
        # node does not exists
        else:
            tree[parent] = [(child, weight)]
    return tree


def pd_z0():
    return max(np.random.normal(alpha_0, np.sqrt(variance_0)), 1)


def cpd(z, t):
    mean = gamma_0 + (alpha * t) + (beta * z) + (gamma * t * z)
    var = variance * t
    return max(np.random.normal(mean, np.sqrt(var)), 1)


def simulation(root: int, graph: dict, alpha0, variance0, alpha, beta, variance) -> dict:
    """
    BFS
    returns z value for all 407 nodes
    """
    gene_lengths = {}
    
    z0 = max(np.random.normal(alpha0, np.sqrt(variance0)), 1)
    queue = [(root, z0)]

    while queue:
        # handle the next element in queue
        node, z = queue.pop(0)
        
        # set node in results in genelength z
        gene_lengths[node] = z
        
        # get all children for the node
        children = graph.get(node, [])

        if not children:
            continue
        else:
            # for all the children for this node calculate their z
            # since we know it's a tree, only the parent can have influence
            for child in children:
                id, t = child
                t = round(t, 3)
                
                # draw a sample
                mean = (alpha * t) + (beta * z)
                var = variance * t
                
                    
                cpd_z = max(np.random.normal(mean, np.sqrt(var)), 1)
                #print(f"id:{id}, t:{t}, var:{var}, std:{np.sqrt(var)}, mean:{mean}, cpd:{cpd_z}")
                #if id == 205:
                #    print(var)
                #    print(mean)
                #    print(cpd_z)
                # append child to queue
                queue.append((id, cpd_z))

    return gene_lengths


def n_simulations(n, root, graph, alpha0, variance0, alpha, beta, variance) -> tuple:
    """
    Returns results for X=X1 ... X204 n times, and y=Z0 n times s
    """
    X = np.empty((n, 204))
    Z = np.empty((n, 203))
    y = np.empty(n)
    
    for i in range(n):
        results = simulation(root=root, graph=graph, alpha0=alpha0[i], variance0=variance0[i], alpha=alpha[i], beta=beta[i], variance=variance[i])
        
        # extract the first 204 values from the dictionary and add them to X as a row
        row_X = [results[key] for key in range(1, 205)]
        row_Z = [results[key] for key in range(205, 408)]
        
        X[i] = np.array(row_X)
        Z[i] = np.array(row_Z)
        
        
    return X, Z



def dfs_paths_with_cpd(graph, start_node, z, path=None):
    # returns all paths and the gene length at each node in the path
    if path is None:
        path = [(start_node, z)]

    if start_node not in graph:
        return [path]

    paths = []
    for child, t in graph[start_node]:
        if child not in [node for node, _ in path]:
            new_z = cpd(z, t)
            child_paths = dfs_paths_with_cpd(graph, child, new_z, path + [(child, new_z)])
            paths.extend(child_paths)

    return paths

In [4]:
# load data
tree_data = pd.read_csv('data/tree.csv')
genes_data = pd.read_csv('data/vert_genes.csv')

# remove root node, since we already have reference to it from else where. unless issue with t?
tree_data = tree_data[tree_data['Parent'].notna()]

# convert parent col to ints
tree_data['Parent'] = tree_data['Parent'].astype(int)

In [5]:
# gamma_0 + (alpha * time) (beta * parent gene len) + (gamma * time * parent gene len)

# Parameters
# for Z_0
size = 5000
alpha0 = np.random.normal(50000, 50, size=size)
variance0 = np.random.normal(5000, 10, size=size)

# for Z_i and X_i
alpha = np.random.normal(0, 0.5, size=size)
beta = np.random.normal(3, 1, size=size)
variance = np.random.normal(2500, 10, size=size)

# Settings
root = 407

# create a dict graphe
graph = create_graph(tree_data)

In [6]:
print(alpha0.shape)
print(variance)
np.min(variance),np.max(variance),np.mean(variance),np.var(variance)

(5000,)
[2507.95599739 2496.55227645 2494.44576974 ... 2512.51185711 2500.81716903
 2502.6206323 ]


(2463.2606656378653, 2534.23100947955, 2500.0736038337445, 99.31103507879153)

In [7]:
# Create data
t0 = time.time()

X, Z = n_simulations(size, root, graph, alpha0, variance0, alpha, beta, variance)
runtime = time.time() - t0
print(f"{runtime:.10f} sec")

11.1944770813 sec


In [8]:
V = np.concatenate((X,Z),axis=1)
print(V)
#V = np.concatenate(X,Z,alpha0,variance0,alpha,beta,variance)

[[3.14137273e+09 3.14137238e+09 1.70043061e+09 ... 3.67197424e+06
  1.98754724e+06 5.00004684e+04]
 [3.56675415e+14 3.56675415e+14 1.01140763e+14 ... 3.40123002e+08
  9.64471151e+07 5.01501600e+04]
 [1.45199501e+06 1.45168927e+06 1.20364187e+06 ... 1.84796413e+05
  1.53144510e+05 4.99598036e+04]
 ...
 [7.18579284e+09 7.18579251e+09 3.71488886e+09 ... 5.06050771e+06
  2.61607663e+06 4.99934032e+04]
 [2.83604683e+11 2.83604683e+11 1.19543499e+11 ... 2.11231344e+07
  8.90383762e+06 5.00473885e+04]
 [7.01081420e+10 7.01081425e+10 3.19344953e+10 ... 1.22492379e+07
  5.57958310e+06 4.99607803e+04]]


In [9]:
# lin r
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

regAlpha = LinearRegression().fit(V, alpha)
regBeta = LinearRegression().fit(V, beta)
regSigma = LinearRegression().fit(V, variance)

# Train results
yt_pred = regAlpha.predict(V)
train_r2 = regAlpha.score(V, alpha)
train_mse = mean_squared_error(alpha, yt_pred)

yt_pred1 = regBeta.predict(V)
yt_pred2 = regSigma.predict(V)
'''# Test results
y_pred = reg.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)'''

print(f"TRAIN. r2: {train_r2:.3f}, MSE: {train_mse:.3f}")
#print(f"TEST. r2: {test_r2:.3f}, MSE: {test_mse:.3f}")

TRAIN. r2: 0.006, MSE: 0.250


In [10]:
print(yt_pred)
print(alpha)

print(regBeta.score(V,beta))
print(regSigma.score(V,variance))
print((yt_pred1-beta))
print((yt_pred2,variance))
print(np.min(yt_pred2))
print(np.min(variance))

[ 0.0043155  -0.01959604  0.01114266 ...  0.0026513  -0.00252054
 -0.00148036]
[-0.0987917  -0.33863532 -0.49135237 ...  0.66877973  0.15390939
  1.05729465]
0.9054314112925632
0.004374907558999719
[-0.06076161 -0.19112354  0.23316474 ... -0.04686099  0.09508501
  0.04230185]
(array([2499.84524855, 2500.3631297 , 2500.20747693, ..., 2499.76313883,
       2499.54257903, 2499.57524023]), array([2507.95599739, 2496.55227645, 2494.44576974, ..., 2512.51185711,
       2500.81716903, 2502.6206323 ]))
2486.3828413449646
2463.2606656378653
